In [1]:
from bs4 import BeautifulSoup
import requests

def generate_url(page: int) -> str:
    return f"https://freedom.pl/mieszkania/kolobrzeg/zp/page/{page}/?km=50"

def get_soup(url: str) -> BeautifulSoup:
    response = requests.get(url)
    return BeautifulSoup(response.content, "html.parser")

In [2]:
flats_urls = set()

for page in range(1, 7):
    url = generate_url(page)
    soup = get_soup(url)

    flats = soup.find_all("div", class_="offert")
    for flat in flats:
        url = flat.find("a")["href"]
        flats_urls.add(url)

In [3]:
import pandas as pd

df1 = pd.DataFrame(flats_urls)
df1

,0
0,https://freedom.pl/oferta/mieszkanie-na-sprzed...
1,https://freedom.pl/oferta/mieszkanie-na-sprzed...
2,https://freedom.pl/oferta/mieszkanie-na-sprzed...
3,https://freedom.pl/oferta/mieszkanie-na-sprzed...
4,https://freedom.pl/oferta/mieszkanie-na-sprzed...
...,...
88,https://freedom.pl/oferta/mieszkanie-na-sprzed...
89,https://freedom.pl/oferta/mieszkanie-na-sprzed...
90,https://freedom.pl/oferta/mieszkanie-na-sprzed...
91,https://freedom.pl/oferta/mieszkanie-na-sprzed...


In [6]:
df1.to_csv('./data/flats_urls.csv', index=False, header=False)

In [7]:
def format_text(text: str) -> str:
    # text = text.lower()
    text = text.strip()
    text = text.replace("\n", "")
    text = text.replace(":", "")
    return text


def fetch_flat(row):
    url = row[0]
    soup = get_soup(url)
    details = soup.find_all("div", class_="detail")

    offer_table = {
        "url": url,
    }

    for detail in details:
        fields = detail.find_all("li")
        for field in fields:
            key = field.find("strong").text
            key = format_text(key)

            value = field.find("small").text
            value = format_text(value)

            offer_table[key] = value

    offer_table["opis"] = soup.find("div", class_="desc-tab").text.strip()

    return offer_table


df2 = df1.apply(fetch_flat, axis=1, result_type="expand")
df2

,url,Rynek,Cena,Powierzchnia,Cena za m2,Liczba pokoi,Piętro,Liczba pięter,Czynsz administracyjny,Typ mieszkania,...,Powierzchnia przedpokoju,Powierzchnia kuchni,Wysokość pomieszczeń (cm),Liczba oddzielnych toalet,Stan mieszkania,Ciepła woda,Rodzaj ogrzewania,Typ okien,Alarm,Klimatyzacja
0,https://freedom.pl/oferta/mieszkanie-na-sprzed...,Wtórny,756 000 zł,47.88 m2,15 789 zł,2,2,4,930,Apartament,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://freedom.pl/oferta/mieszkanie-na-sprzed...,Wtórny,400 000 zł,43.2 m2,9 259 zł,2,4,4,710,Rozkładowe,...,3.4,"3,8 m2",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://freedom.pl/oferta/mieszkanie-na-sprzed...,Wtórny,475 000 zł,32.75 m2,14 504 zł,2,4,6,290,Apartament,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://freedom.pl/oferta/mieszkanie-na-sprzed...,Wtórny,475 000 zł,68.5 m2,6 934 zł,3,4,4,770,Rozkładowe,...,NaN,7 m2,260,1,NaN,NaN,NaN,NaN,NaN,NaN
4,https://freedom.pl/oferta/mieszkanie-na-sprzed...,Wtórny,550 000 zł,41.81 m2,13 155 zł,2,1,4,544.25,Jednopoziomowe,...,NaN,NaN,260,NaN,Bardzo dobry,Wodociąg miejski,CO miejskie,PCV,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,https://freedom.pl/oferta/mieszkanie-na-sprzed...,Wtórny,950 000 zł,103.38 m2,9 189 zł,4,6,7,1342,Rozkładowe,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,https://freedom.pl/oferta/mieszkanie-na-sprzed...,Wtórny,352 440 zł,32.04 m2,11 000 zł,1,0,4,504.99,"Apartament ,Jednopoziomowe",...,NaN,NaN,255,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,https://freedom.pl/oferta/mieszkanie-na-sprzed...,Wtórny,1 090 000 zł,52 m2,20 962 zł,2,1,5,NaN,Apartament,...,NaN,NaN,NaN,NaN,Wysoki standard,Wodociąg miejski,CO miejskie,Aluminiowe,Tak,Tak
91,https://freedom.pl/oferta/mieszkanie-na-sprzed...,Wtórny,620 000 zł,36.82 m2,16 839 zł,2,5,6,250,Jednopoziomowe,...,NaN,NaN,270,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df2.to_pickle('./data/flats_dirty.pkl')

In [10]:
df3 = df2[["url", "Numer oferty"]]
df3

,url,Numer oferty
0,https://freedom.pl/oferta/mieszkanie-na-sprzed...,28111/3685/OMS
1,https://freedom.pl/oferta/mieszkanie-na-sprzed...,28902/3685/OMS
2,https://freedom.pl/oferta/mieszkanie-na-sprzed...,28987/3685/OMS
3,https://freedom.pl/oferta/mieszkanie-na-sprzed...,26218/3685/OMS
4,https://freedom.pl/oferta/mieszkanie-na-sprzed...,31133/3685/OMS
...,...,...
88,https://freedom.pl/oferta/mieszkanie-na-sprzed...,28798/3685/OMS
89,https://freedom.pl/oferta/mieszkanie-na-sprzed...,27282/3685/OMS
90,https://freedom.pl/oferta/mieszkanie-na-sprzed...,29015/3685/OMS
91,https://freedom.pl/oferta/mieszkanie-na-sprzed...,26784/3685/OMS


In [11]:
import os

def download_photos(row):
    nr_oferty = row["Numer oferty"]
    url = row["url"]

    soup = get_soup(url)

    images = soup.find("div", class_="swiper-container").find_all("img")
    for image in images:
        src = image["src"].split("-")[:-1]
        photo_url = "-".join(src) + ".jpg"

        filename = photo_url.split("/")[-1]

        directory = f"./images/{nr_oferty}"
        os.makedirs(directory, exist_ok=True)

        filepath = f"{directory}/{filename}"
        if not os.path.isfile(filepath):
            response = requests.get(photo_url)
            with open(filepath, 'wb') as file:
                file.write(response.content)
        else:
            print(f"File {filepath} already exists")

    return row

df3.apply(download_photos, axis=1)

File ./images/28111/3685/OMS/95890217.jpg already exists
File ./images/28111/3685/OMS/95890219.jpg already exists
File ./images/28111/3685/OMS/95890221.jpg already exists
File ./images/28111/3685/OMS/95892970.jpg already exists
File ./images/28111/3685/OMS/95892995.jpg already exists
File ./images/28111/3685/OMS/95890222.jpg already exists
File ./images/28111/3685/OMS/95890225.jpg already exists
File ./images/28111/3685/OMS/95890234.jpg already exists
File ./images/28111/3685/OMS/95890229.jpg already exists
File ./images/28111/3685/OMS/95890237.jpg already exists
File ./images/28111/3685/OMS/95890238.jpg already exists
File ./images/28111/3685/OMS/95946892.jpg already exists
File ./images/28111/3685/OMS/95890239.jpg already exists
File ./images/28111/3685/OMS/95890241.jpg already exists
File ./images/28111/3685/OMS/95890244.jpg already exists
File ./images/28111/3685/OMS/95890247.jpg already exists
File ./images/28111/3685/OMS/95890250.jpg already exists
File ./images/28111/3685/OMS/95

,url,Numer oferty
0,https://freedom.pl/oferta/mieszkanie-na-sprzed...,28111/3685/OMS
1,https://freedom.pl/oferta/mieszkanie-na-sprzed...,28902/3685/OMS
2,https://freedom.pl/oferta/mieszkanie-na-sprzed...,28987/3685/OMS
3,https://freedom.pl/oferta/mieszkanie-na-sprzed...,26218/3685/OMS
4,https://freedom.pl/oferta/mieszkanie-na-sprzed...,31133/3685/OMS
...,...,...
88,https://freedom.pl/oferta/mieszkanie-na-sprzed...,28798/3685/OMS
89,https://freedom.pl/oferta/mieszkanie-na-sprzed...,27282/3685/OMS
90,https://freedom.pl/oferta/mieszkanie-na-sprzed...,29015/3685/OMS
91,https://freedom.pl/oferta/mieszkanie-na-sprzed...,26784/3685/OMS
